<a href="https://colab.research.google.com/github/EmilisGit/Deep_learning/blob/main/finetuning_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
transformers.__version__

'4.46.2'

# 1. Duomenų paruošimas

Atliksime sentimentų analizę, naudojant iš anksto apmokytą BERT modelį.

Naudosime duomenis iš [Yelp Open Dataset](https://www.yelp.com/dataset). Tai bus atsiliepimai apie maisto restoranus.

In [24]:
!pip install datasets -q

from datasets import load_dataset

dataset = load_dataset("antash420/text-summarization-alpaca-format")

README.md:   0%|          | 0.00/647 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/239M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/268M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/245M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/68.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/59.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [25]:
for i in range(0,5):
  print(dataset['train'][i])

{'input': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart.

In [28]:
tokenizer = AutoTokenizer.from_pretrained("google/t5-efficient-mini")
tokenizer('Example sentence to tokenize')

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

{'input_ids': [18792, 7142, 12, 14145, 1737, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]}

Kiek turime apmokymo duomenų (mėginių)?

In [35]:
tokens = tokenizer(dataset['train'][0]['input'])
print(len(tokens['input_ids']))

636


In [6]:
print("Apmokymo duomenu: ", len(dataset['train']))
print("Testavimo duomenu: ", len(dataset['test']))

Apmokymo duomenu:  650000
Testavimo duomenu:  50000


Toliau naudosime tik dalį atsiliepimų viso proceso greitinimui:

In [7]:
n_samples = 2000
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(n_samples))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(n_samples))
len(small_train_dataset)

2000

In [8]:
def tokenize(text):
  return tokenizer(text['text'], truncation=True)

encoded_train = small_train_dataset.map(tokenize, batched=True)
encoded_eval = small_eval_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Pastebekime kad po tokenizavimo mūsų `Dataset` objectas gavo naujas savybes (stulpelius, raktažodžius) - `'input_ids'`, `'token_type_ids'` ir `'attention_mask'`:

In [9]:
encoded_train.features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

Paruošiame klasifikavimo žymių žodynus:

In [10]:
id2label = {0 : "1 star", 1 : "2 star", 2 : "3 star", 3 : "4 star", 4 : "5 star"}
label2id = {"1 star" : 0, "2 star" : 1, "3 star" : 2, "4 star" : 3, "5 star" : 4}

## 2. Modelio inicializavimas ir derinimas

In [11]:
from transformers import TFAutoModelForSequenceClassification

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-efficient-mini")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Ką tik gautas įspėjimas apie kai kuriuos išmestus svorius yra normalus - šiuo atveju mes pašalinome modelio bloką, atsakingą už maskuotą kalbos modeliavimą - tai yra įprasta BERT modelio užduotis, o mes norime atlikti sentimentų analizę (teksto klasifikavimą), dėl ko mums ir reikia papildomas modelio derinimas.

Norime įkelsti mūsų duomenis kaip Tensorflow duomenų rinkinius - tai bus padaryta automatiškai ir atitiks modelio architektūrą:

In [17]:
!pip install datasets -q

In [18]:
batch_size = 16

tf_train_dataset = model.prepare_tf_dataset(
    encoded_train,
    shuffle=True,
    batch_size=batch_size,
    tokenizer=tokenizer
)

tf_eval_dataset = model.prepare_tf_dataset(
    encoded_eval,
    shuffle=True,
    batch_size=batch_size,
    tokenizer=tokenizer
)

ImportError: 
TFDistilBertForSequenceClassification requires the 🤗 Datasets library but it was not found in your environment. You can install it with:
```
pip install datasets
```
In a notebook or a colab, you can install it by executing a cell with
```
!pip install datasets
```
then restarting your kernel.

Note that if you have a local folder named `datasets` or a local python file named `datasets.py` in your current
working directory, python may try to import this instead of the 🤗 Datasets library. You should rename this folder or
that python file if that's the case. Please note that you may need to restart your runtime after installation.


Visi `transformers` modeliai pagal nutylėjimą gali patys pasirinkti protingą nuostolių funkciją, todėl mums nereikis dėti `loss` parametro į `compile()`. Tuo metu `transformers` rekomenduoja sukonstruoti `AdamW` tipo optimizatorių patiems:

In [19]:
num_epochs = 3

batches_per_epoch = len(encoded_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = transformers.create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)

In [20]:
model.compile(optimizer=optimizer, metrics=['accuracy'])

Laikas fittinti modelį:

In [ ]:
model.fit(
    ...,
    validation_data=...,
    epochs=num_epochs
)

Epoch 1/3
125/125 [==============================] - 158s 1s/step - loss: 1.6060 - accuracy: 0.2190 - val_loss: 1.5970 - val_accuracy: 0.2445
Epoch 2/3
125/125 [==============================] - 139s 1s/step - loss: 1.5647 - accuracy: 0.2675 - val_loss: 1.5755 - val_accuracy: 0.2495
Epoch 3/3
125/125 [==============================] - 140s 1s/step - loss: 1.4898 - accuracy: 0.3100 - val_loss: 1.5211 - val_accuracy: 0.2985


## 3. Modelio panaudojimas klasifikavimui

In [ ]:
sentences = [...]
tokenized = tokenizer(sentences, return_tensors="np", padding="longest",
                      return_token_type_ids=False)

predicted_logits = model(tokenized).logits
classifications = ...